In [15]:
# Importing Dependencies
import pandas as pd


In [16]:
# Importing Files
steam = "Original Files/steam.csv"
steam_media_data = "Original Files/steam_media_data.csv"
steam_support_info = "Original Files/steam_support_info.csv"
steam_tag = "Original Files/steamspy_tag_data.csv"

Games = pd.read_csv(steam)
steam_media_data_df = pd.read_csv(steam_media_data)
steam_support_info_df = pd.read_csv(steam_support_info)
steam_tag_df = pd.read_csv(steam_tag)

In [17]:
# Shape of the DF
Games.shape

(27075, 18)

In [21]:
# Games DF
Games.to_csv('Files/Games.csv')

In [85]:
Games.head(1000)

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,65780,ARMA: Gold Edition,2011-07-13,1,Bohemia Interactive,Bohemia Interactive,windows,16,Single-player;Multi-player,Action;Simulation;Strategy,Simulation;Action;Military,0,548,209,22,22,500000-1000000,5.99
996,65790,ARMA: Cold War Assault,2011-08-08,1,Bohemia Interactive,Bohemia Interactive,windows,16,Single-player,Action;Simulation;Strategy,Simulation;Action;Strategy,0,3195,942,28,39,1000000-2000000,3.49
997,65800,Dungeon Defenders,2011-10-18,1,Trendy Entertainment,Trendy Entertainment,windows;mac;linux,0,Single-player;Multi-player;Co-op;Shared/Split ...,Action;Indie;RPG;Strategy,Tower Defense;RPG;Co-op,114,10693,932,1993,1205,1000000-2000000,9.99
998,65930,The Bureau: XCOM Declassified,2013-08-22,1,2K Marin,2K,windows;mac,16,Single-player;Steam Achievements;Full controll...,Action,Action;Tactical;Sci-fi,41,6661,2923,201,168,2000000-5000000,14.99


In [63]:
# Languages DF
languages =  Games.loc[:,['english']].head() 
languages.columns = ['language']
languages = languages.drop(
    labels=["language"],
    axis=1,
) 
languages["languageid"] = ""
languages["language"] = ""
languages.head()

English = {'languageid':'1','language':'English'}
languages = languages.append(English, ignore_index=True)

languages.drop(languages.index[[0,1,2,3,4]])
languages.reset_index(drop=True, inplace=True)
languages

languages.to_csv('Files/Languages.csv')

In [96]:
# Developers DF
developers =  Games.loc[:,['developer']]
developers.columns = ['developers']

developers = developers.groupby(['developers'])
developers_df = developers.count()

developers_df["developersid"] = ""
developers_df.set_index('developersid')



developers_df.head()

developers_df.to_csv('Files/Developers.csv')

In [97]:
# Publishers DF
publishers =  Games.loc[:,['publisher']] 
publishers.columns = ['publishers']

publishers = publishers.groupby(['publishers'])
publishers_df = publishers.count()

publishers_df["publishersid"] = ""
publishers_df.set_index('publishersid')



publishers_df.head()
publishers_df.to_csv('Files/Publishers.csv')

In [98]:
# Owners DF
Owners =  Games.loc[:,['owners']]
Owners.columns = ['owners']

Owners = Owners.groupby(['owners'])
Owners_df = Owners.count()

Owners_df["ownersid"] = ""
Owners_df.set_index('ownersid')



Owners_df.head()
Owners_df.to_csv('Files/Owners.csv')

In [90]:
appid = Games.loc[:,['appid']]
appid

,appid
0,10
1,20
2,30
3,40
4,50
...,...
27070,1065230
27071,1065570
27072,1065650
27073,1066700


In [99]:
# Categories Joint
game_categories = Games.loc[:,['appid']]
game_categories

game_categories.to_csv('Files/game_categories.csv')

In [100]:
# Platform Joint
game_platform = Games.loc[:,['appid']]
game_platform

game_platform.to_csv('Files/game_platform.csv')

In [101]:
# Platform Tag
game_tag = Games.loc[:,['appid']]
game_tag

game_tag.to_csv('Files/game_tag.csv')

In [102]:
# Genres Joint
game_genres = Games.loc[:,['appid']]
game_genres

game_genres.to_csv('Files/game_genres.csv')

In [ ]:
# Categories



In [ ]:
# Platforms

In [ ]:
# Tags

In [ ]:
# Genres